In [1]:
import pandas as pd
import re 
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
import pickle
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
# import en_core_web_sm
import nltk
import os
from nltk.tokenize import word_tokenize
import emoji

C:\Users\Annelie Schridde\AppData\Roaming\Python\Python38\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [1]:
#import sys
#sys.path.append('../')
#import functions
#exec(open('./functions.py').read())
%run functions.py

C:\Anaconda\envs\uni\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\home\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\home\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\home\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
... OK (Got response in 0.42 seconds)
Writing emoji data to C:\Users\home\.demoji\codes.json ...
... OK


In [2]:
df = pickle.load(open('data_n_models/de_df_filtered.pkl', "rb"))
de_df = df[df.language == 'de'].reset_index()
de_df.head()

,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,favorite_count,in_reply_to_status_id,keyword,language
0,1,1079317443523297280,12303,Boulder667,"@spdde @LadyBitchRay1 Viel Spaß, damit eine Wa...",0,"Liverpool, England",07-Mar-2021,116,False,1.0,1.368148e+18,zalando,de
1,4,310342425,11476,AndyBaldauf,@dicecco @Zalando aber erst wenn sie wieder dü...,0,Thurgau | St. Gallen | Zürich,06-Mar-2021,1848,False,0.0,1.368146e+18,zalando,de
2,10,793418126532734976,521,MILESmobility,Good News! Alle interessierten MitarbeiterInne...,0,"Berlin, Deutschland",05-Mar-2021,766,False,8.0,NaN,zalando,de
3,11,1119231287078858752,23,pici1303,@Zalando Ich weiß dass der Schein da rein muss...,0,,04-Mar-2021,0,False,0.0,1.367531e+18,zalando,de
4,12,1119231287078858752,23,pici1303,@Zalando Ich hab mich nur gefragt wofür dieser...,0,,04-Mar-2021,0,False,0.0,1.367528e+18,zalando,de


In [10]:
def clean_complete(tweet):
    """
    tweet: pandas series
    prepares tweets complete cleaning for further lemmatization and dering embeddings
    """
    pat = r"(\\n)|(@\w*)|((www\.[^\s]+)|(https?://[^\s]+))"
    tweet = tweet.str.replace(pat, '')

    #remove repeated charachters
    
    #replace emoticons with words
    #SMILEYS = {":-(":"sad", ":‑)":"smiley", ":-P":"playfullness", ":-/":'confused'}

    tweet = tweet.str.replace(r':-\)', ' smile')
    tweet = tweet.str.replace(r':-\(', ' sad')
    tweet = tweet.str.replace(r':-\/', ' confused')
    tweet = tweet.str.replace(r':-P', ' playfullness')

    #delete \xa
    tweet = tweet.str.replace('\xa0', '')

    tweet = tweet.str.replace('&amp', '')
    tweet = tweet.str.replace('\n', '')
    tweet = tweet.str.replace('"', '')
    #to lower case
    tweet = tweet.str.lower()

    #covert hashtags to the normal text
    tweet = tweet.str.replace(r'#([^\s]+)', r'\1')

    #delete numbers
    tweet = [strip_numeric(c) for c in tweet]

    #replacing emojies with descriptions '❤️-> red heart'
    #tweet = [demoji.replace_with_desc(c, ' ') for c in tweet]

    #delete punctuation
    tweet = [strip_punctuation(c) for c in tweet]

    #remove stop words
    tweet = [remove_stopwords(c) for c in tweet]

    #remove short words
    tweet = [strip_short(c) for c in tweet]

    #remove mult whitespaces
    tweet = [strip_multiple_whitespaces(c) for c in tweet]
    return tweet


In [11]:
def clean_vader(tweet):
    """
    tweet: pandas series
    prepares tweets for vader sentiment analysis
    """

    pat = r"(\\n)|(@\w*)|((www\.[^\s]+)|(https?://[^\s]+))"
    tweet = tweet.str.replace(pat, '')

    #replace emoticons with words
    #SMILEYS = {":-(":"sad", ":‑)":"smiley", ":-P":"playfullness", ":-/":'confused'}

    #tweet = tweet.str.replace(r':-\)', ' smile')
    #tweet = tweet.str.replace(r':-\(', ' sad')
    #tweet = tweet.str.replace(r':-\/', ' confused')
    #tweet = tweet.str.replace(r':-P', ' playfullness')

    #delete \xa
    tweet = tweet.str.replace('\xa0', '')

    tweet = tweet.str.replace('&amp', '')
    tweet = tweet.str.replace('\n', '')

    #to lower case
    #tweet = tweet.str.lower()

    #covert hashtags to the normal text
    tweet = tweet.str.replace(r'#([^\s]+)', r'\1')

    #delete numbers
    tweet = [strip_numeric(c) for c in tweet]

    #replacing emojies with descriptions '❤️-> red heart'
    #tweet = [demoji.replace_with_desc(c, ' ') for c in tweet]

    #delete punctuation
    #tweet = [strip_punctuation(c) for c in tweet]

    #remove stop words
    #tweet = [remove_stopwords(c) for c in tweet]

    #remove short words
    tweet = [strip_short(c) for c in tweet]

    #remove mult whitespaces
    tweet = [strip_multiple_whitespaces(c) for c in tweet]
    return tweet


In [3]:
#import demoji
de_df['clean'] = clean_complete(de_df.tweet)
de_df.head()

c:\Users\home\Documents\GitHub\Zalando_Analytics_HWR_BIPM\functions.py:58: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet = tweet.str.replace(pat, '')
c:\Users\home\Documents\GitHub\Zalando_Analytics_HWR_BIPM\functions.py:65: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet = tweet.str.replace(r':-\)', ' smile')
c:\Users\home\Documents\GitHub\Zalando_Analytics_HWR_BIPM\functions.py:66: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet = tweet.str.replace(r':-\(', ' sad')
c:\Users\home\Documents\GitHub\Zalando_Analytics_HWR_BIPM\functions.py:67: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet = tweet.str.replace(r':-\/', ' confused')
c:\Users\home\Documents\GitHub\Zalando_Analytics_HWR_BIPM\functions.py:80: FutureWarning: The default value of regex will change from True to Fal

,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,favorite_count,in_reply_to_status_id,keyword,language,clean
0,1,1079317443523297280,12303,Boulder667,"@spdde @LadyBitchRay1 Viel Spaß, damit eine Wa...",0,"Liverpool, England",07-Mar-2021,116,False,1.0,1.368148e+18,zalando,de,viel spaß damit eine wahl gewinnen stellt euch...
1,4,310342425,11476,AndyBaldauf,@dicecco @Zalando aber erst wenn sie wieder dü...,0,Thurgau | St. Gallen | Zürich,06-Mar-2021,1848,False,0.0,1.368146e+18,zalando,de,aber erst wenn sie wieder dürfen playfullness
2,10,793418126532734976,521,MILESmobility,Good News! Alle interessierten MitarbeiterInne...,0,"Berlin, Deutschland",05-Mar-2021,766,False,8.0,NaN,zalando,de,good news alle interessierten mitarbeiterinnen...
3,11,1119231287078858752,23,pici1303,@Zalando Ich weiß dass der Schein da rein muss...,0,,04-Mar-2021,0,False,0.0,1.367531e+18,zalando,de,ich weiß dass der schein rein muss aber ist mi...
4,12,1119231287078858752,23,pici1303,@Zalando Ich hab mich nur gefragt wofür dieser...,0,,04-Mar-2021,0,False,0.0,1.367528e+18,zalando,de,ich hab mich nur gefragt wofür dieser extra co...


In [4]:
de_df['clean_vader'] = clean_vader(de_df.tweet)
de_df.head()

c:\Users\home\Documents\GitHub\Zalando_Analytics_HWR_BIPM\functions.py:108: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet = tweet.str.replace(pat, '')
c:\Users\home\Documents\GitHub\Zalando_Analytics_HWR_BIPM\functions.py:128: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet = tweet.str.replace(r'#([^\s]+)', r'\1')


,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,favorite_count,in_reply_to_status_id,keyword,language,clean,clean_vader
0,1,1079317443523297280,12303,Boulder667,"@spdde @LadyBitchRay1 Viel Spaß, damit eine Wa...",0,"Liverpool, England",07-Mar-2021,116,False,1.0,1.368148e+18,zalando,de,viel spaß damit eine wahl gewinnen stellt euch...,"Viel Spaß, damit eine Wahl gewinnen. Stellt Eu..."
1,4,310342425,11476,AndyBaldauf,@dicecco @Zalando aber erst wenn sie wieder dü...,0,Thurgau | St. Gallen | Zürich,06-Mar-2021,1848,False,0.0,1.368146e+18,zalando,de,aber erst wenn sie wieder dürfen playfullness,aber erst wenn sie wieder dürfen :-P
2,10,793418126532734976,521,MILESmobility,Good News! Alle interessierten MitarbeiterInne...,0,"Berlin, Deutschland",05-Mar-2021,766,False,8.0,NaN,zalando,de,good news alle interessierten mitarbeiterinnen...,Good News! Alle interessierten MitarbeiterInne...
3,11,1119231287078858752,23,pici1303,@Zalando Ich weiß dass der Schein da rein muss...,0,,04-Mar-2021,0,False,0.0,1.367531e+18,zalando,de,ich weiß dass der schein rein muss aber ist mi...,Ich weiß dass der Schein rein muss aber was is...
4,12,1119231287078858752,23,pici1303,@Zalando Ich hab mich nur gefragt wofür dieser...,0,,04-Mar-2021,0,False,0.0,1.367528e+18,zalando,de,ich hab mich nur gefragt wofür dieser extra co...,Ich hab mich nur gefragt wofür dieser extra Co...


In [14]:
print(de_df.tweet[2])
print(de_df.clean[2])
print(de_df.clean_vader[2])

Good News! Alle interessierten MitarbeiterInnen der Berliner @ChariteBerlin erhalten täglich ein #Mobilitätsbudget in Höhe von 10€, das sie für Fahrten mit #MILES einlösen können.

Danke an @Zalando, die die ganze Aktion sponsern! https://t.co/MVSJDv8j2i
good news alle interessierten mitarbeiterinnen der berliner erhalten täglich ein mobilitätsbudget höhe von das sie für fahrten mit miles einlösen können danke die die ganze aktion sponsern
Good News! Alle interessierten MitarbeiterInnen der Berliner erhalten täglich ein Mobilitätsbudget Höhe von das sie für Fahrten mit MILES einlösen können.Danke die die ganze Aktion sponsern!


In [5]:
#Vater sentiment analyzer
#The Compound score is a metric that calculates the sum of all the lexicon ratings which have been normalized between -1(most extreme negative) and +1 (most extreme positive).

sid_obj = SentimentIntensityAnalyzer()
de_df['compound'] = [sid_obj.polarity_scores(c)['compound'] for c in de_df['clean_vader']]

In [16]:
print(de_df[['clean_vader', 'compound']].describe())

          compound
count  2709.000000
mean     -0.159865
std       0.372277
min      -0.966100
25%      -0.599400
50%       0.000000
75%       0.000000
max       0.969800


# Testing cleaning function

In [17]:
#testing
#delete links, users
pat = r"(\\n)|(@\w*)|((https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,}))"
de_df.tweet = de_df.tweet.str.replace(pat, '')
de_df.tweet[2]

<ipython-input-17-5463527a300c>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  de_df.tweet = de_df.tweet.str.replace(pat, '')


'Good News! Alle interessierten MitarbeiterInnen der Berliner  erhalten täglich ein #Mobilitätsbudget in Höhe von 10€, das sie für Fahrten mit #MILES einlösen können.\n\nDanke an , die die ganze Aktion sponsern! '

In [18]:
#remove chars more than twice
#str.replace( /(.)\1{2,}/g, '$1$1' )

In [19]:

#testing
# #replace emoticons with words
#SMILEYS = {":-(":"sad", ":‑)":"smiley", ":-P":"playfullness", ":-/":'confused'}

de_df.tweet = de_df.tweet.str.replace(r':-\)', ' smile')
de_df.tweet = de_df.tweet.str.replace(r':-\(', ' sad')
de_df.tweet = de_df.tweet.str.replace(r':-\/', ' confused')
de_df.tweet = de_df.tweet.str.replace(r':-P', ' playfullness')

de_df.tweet[2]

<ipython-input-19-a5a60d69cb5d>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  de_df.tweet = de_df.tweet.str.replace(r':-\)', ' smile')
<ipython-input-19-a5a60d69cb5d>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  de_df.tweet = de_df.tweet.str.replace(r':-\(', ' sad')
<ipython-input-19-a5a60d69cb5d>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  de_df.tweet = de_df.tweet.str.replace(r':-\/', ' confused')


'Good News! Alle interessierten MitarbeiterInnen der Berliner  erhalten täglich ein #Mobilitätsbudget in Höhe von 10€, das sie für Fahrten mit #MILES einlösen können.\n\nDanke an , die die ganze Aktion sponsern! '

In [20]:
#testing#delete \xa
de_df.tweet = de_df.tweet.str.replace('\xa0', '')
de_df.tweet[0]

'  Viel Spaß, damit eine Wahl zu gewinnen. Stellt Euch mal eine   vor, die sich um echte Probleme kümmert...zum Beispiel, wie es sein kann das Luxemburg und Irland Beihilfe zur Steuerhinterziehung, zu Lasten der deutschen Steuerzahler begeht. Und Firmen wie  '

In [22]:
#testing
de_df.tweet = de_df.tweet.str.replace('&amp', '')
de_df.tweet = de_df.tweet.str.replace('\n', '')
de_df.tweet[0]


'  Viel Spaß, damit eine Wahl zu gewinnen. Stellt Euch mal eine   vor, die sich um echte Probleme kümmert...zum Beispiel, wie es sein kann das Luxemburg und Irland Beihilfe zur Steuerhinterziehung, zu Lasten der deutschen Steuerzahler begeht. Und Firmen wie  '

In [23]:
#testing#to lower case
de_df.tweet = de_df.tweet.str.lower()
de_df.tweet[2]

'good news! alle interessierten mitarbeiterinnen der berliner  erhalten täglich ein #mobilitätsbudget in höhe von 10€, das sie für fahrten mit #miles einlösen können.danke an , die die ganze aktion sponsern! '

In [24]:
#testing#covert hashtags to the normal text
de_df.tweet = de_df.tweet.str.replace(r'#([^\s]+)', r'\1')
de_df.tweet[2]

<ipython-input-24-46b7def794f7>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  de_df.tweet = de_df.tweet.str.replace(r'#([^\s]+)', r'\1')


'good news! alle interessierten mitarbeiterinnen der berliner  erhalten täglich ein mobilitätsbudget in höhe von 10€, das sie für fahrten mit miles einlösen können.danke an , die die ganze aktion sponsern! '

In [25]:
#testing#delete numbers
de_df.tweet = [strip_numeric(c) for c in de_df.tweet]
de_df.tweet[7]

'  das ist echt zum mäusemelken'

In [26]:
#testing#replacing emojies with descriptions '❤️-> red heart'
de_df.tweet = [demoji.replace_with_desc(c, ' ') for c in de_df.tweet]
de_df.tweet[9]

' wie gut, dass ich auch genau das nicht getan habe.'

In [27]:
##testing delete punctuation
de_df.tweet = [strip_punctuation(c) for c in de_df.tweet]
de_df.tweet[7]

'  das ist echt zum mäusemelken'

In [28]:
#testing
de_df.tweet = [remove_stopwords(c) for c in de_df.tweet]
de_df.tweet[7]

'das ist echt zum mäusemelken'

In [29]:
#testin
de_df.tweet = [strip_multiple_whitespaces(c) for c in de_df.tweet]
de_df.tweet[7]

'das ist echt zum mäusemelken'

In [30]:
def lemmatize(tweet):
    '''
    tweet: pandas series
    should be applied on the cleaned tweets to transform words to their initial base form.
    For example: suggests -> suggest, deliveries -> delivery
    '''
    nlp = spacy.load("de_core_news_sm")
    tweet = [nlp(c) for c in tweet]
    tweet = [" ".join([token.lemma_ for token in t]) for t in tweet]
    return tweet

In [7]:
de_df['lemma'] = lemmatize(de_df.clean)
de_df.lemma[7]

'das ist echt zum mäusemelken'

In [8]:
de_df.head()

,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,favorite_count,in_reply_to_status_id,keyword,language,clean,clean_vader,compound,lemma
0,1,1079317443523297280,12303,Boulder667,"@spdde @LadyBitchRay1 Viel Spaß, damit eine Wa...",0,"Liverpool, England",07-Mar-2021,116,False,1.0,1.368148e+18,zalando,de,viel spaß damit eine wahl gewinnen stellt euch...,"Viel Spaß, damit eine Wahl gewinnen. Stellt Eu...",-0.5994,viel spaß damit eine wahl gewinnen stellt euch...
1,4,310342425,11476,AndyBaldauf,@dicecco @Zalando aber erst wenn sie wieder dü...,0,Thurgau | St. Gallen | Zürich,06-Mar-2021,1848,False,0.0,1.368146e+18,zalando,de,aber erst wenn sie wieder dürfen playfullness,aber erst wenn sie wieder dürfen :-P,0.4995,aber erst wenn sie wieder dürfen playfullness
2,10,793418126532734976,521,MILESmobility,Good News! Alle interessierten MitarbeiterInne...,0,"Berlin, Deutschland",05-Mar-2021,766,False,8.0,NaN,zalando,de,good news alle interessierten mitarbeiterinnen...,Good News! Alle interessierten MitarbeiterInne...,-0.7568,good news alle interessierten mitarbeiterinnen...
3,11,1119231287078858752,23,pici1303,@Zalando Ich weiß dass der Schein da rein muss...,0,,04-Mar-2021,0,False,0.0,1.367531e+18,zalando,de,ich weiß dass der schein rein muss aber ist mi...,Ich weiß dass der Schein rein muss aber was is...,0.0000,ich weiß dass der schein rein muss aber ist mi...
4,12,1119231287078858752,23,pici1303,@Zalando Ich hab mich nur gefragt wofür dieser...,0,,04-Mar-2021,0,False,0.0,1.367528e+18,zalando,de,ich hab mich nur gefragt wofür dieser extra co...,Ich hab mich nur gefragt wofür dieser extra Co...,0.0000,ich hab mich nur gefragt wofür dieser extra co...


# Data description
- tweet: initial tweet as it was downloaded from API
- clean: tweet cleaned completely from punctuation, emojies, emoticons, stopwords, special characters, users, hashtags, links
- clean_vader: tweet partly cleaned, saving punctuation, emoticons, emojies for using library Vader to get the sentiment of the tweet
- compouns: from -1 (negative) to 1 (positive), 0 - neutral, a tweet sentiment derived fro, Vader library
- lem: intermidiate step before lemmatization
- lemma: lemmatized words (sent -> send)

# Save the dataframe

In [12]:
pickle.dump(de_df, open(r'.\data_n_models\de_df_labelled.pkl', 'wb'))